In [4]:
from argparse import Namespace
from prafa.universe import Universe
import pandas as pd
import numpy as np
import matplotlib as plt
import pickle

In [6]:
filepath = 'results/portfolio_sp500_lagrange_full_30.json'
with open(filepath, 'rb') as f:
    portfolios = pickle.load(f)
print(portfolios)

{Timestamp('2014-01-02 00:00:00'): {'EIX': np.float64(0.002207505518763797), 'AEE': np.float64(0.002207505518763797), 'AA': np.float64(0.002207505518763797), 'LYB': np.float64(0.002207505518763797), 'AXP': np.float64(0.002207505518763797), 'BA': np.float64(0.002207505518763797), 'CAT': np.float64(0.002207505518763797), 'JPM': np.float64(0.002207505518763797), 'CVX': np.float64(0.002207505518763797), 'KO': np.float64(0.002207505518763797), 'ABBV': np.float64(0.002207505518763797), 'DIS': np.float64(0.002207505518763797), 'XOM': np.float64(0.002207505518763797), 'PSX': np.float64(0.002207505518763797), 'GE': np.float64(0.002207505518763797), 'HPQ': np.float64(0.002207505518763797), 'HD': np.float64(0.002207505518763797), 'IBM': np.float64(0.002207505518763797), 'JNJ': np.float64(0.002207505518763797), 'MCD': np.float64(0.002207505518763797), 'MRK': np.float64(0.002207505518763797), 'MMM': np.float64(0.002207505518763797), 'BAC': np.float64(0.002207505518763797), 'PFE': np.float64(0.00220

In [7]:


args = Namespace()
args.index = 'sp500'
args.data_path = 'financial_data'
args.result_path = 'results'
args.solution_name = 'lagrange_full'
args.rebalancing = 6  
args.start_date = '2014-01-02'
args.end_date = '2025-01-02'

# Initialiser l'objet Universe
universe = Universe(args)


# S'assurer que les dates sont ordonnées
dates = sorted(list(portfolios.keys()))
n = len(dates)

rendements_portefeuille = []
rendements_indice = []
index_dates = []

for i in range(n - 1):  # On va jusqu'à n-1 car on a besoin de t et t+1
    start_date = dates[i]
    end_date = dates[i + 1] - pd.tseries.offsets.BDay(1)  # veille du prochain rebalance

    # Extraire rendements entre deux rebalancements
    universe.new_universe(start_date, end_date, training=False)
    window_returns = universe.get_stocks_returns()
    weights = list(portfolios[start_date].values())

    # Produit matriciel : rendements du portefeuille
    rp = (window_returns @ weights)
    ri = universe.get_index_returns()

    print(f"\n \n \n Pourquoi rp et ri sont vides ????")
    print(len(rp))
    print(len(ri))
    rendements_portefeuille += rp
    rendements_indice += ri
    index_dates += list(window_returns.index)

# Construire les séries temporelles
rendements_portefeuille = pd.Series(rendements_portefeuille, index=index_dates)
rendements_indice = pd.Series(rendements_indice, index=index_dates)

# Cumuler les rendements
rendements_cumules_portefeuille = (rendements_portefeuille + 1).cumprod() -1
rendements_cumules_indice = (rendements_indice + 1).cumprod() - 1

# Tracer
ax = rendements_cumules_portefeuille.plot(label="Portefeuille")
rendements_cumules_indice.plot(ax=ax, label="Indice de référence", linestyle='--')
ax.set_title("Performance cumulée (hors-échantillon)")
ax.set_ylabel("Rendement cumulé")
ax.legend()
plt.grid(True)
plt.show()

⚠️ Les actions suivantes ne sont pas dans les données de rendement : {'1448062D', '9903115D', 'VALPQ', 'BTUUQ', '9983490D', '1684442D', '1284849D', 'BF.B', '1922150D', '2557974D', '1752754D', '1715651D', '2326248D', 'BBBYQ', 'NEBLQ', 'DOFSQ', '9876549D', '1927294D', '2558202D', '1255459D', '1539941D', '9980328D', '1541931D', '1858968D', '9876544D', '1431816D', 'FTRCQ', '1436513D', '1856613D', '2370058D', 'DNRCQ', '1288453D', '1566019D', '2078185D', '9876566D', '2258717D', '1806854D', '1518855D', '9990213D', '1697067D', 'WINMQ', '1831877D', '9991429D', 'BRK.B', '2544554D', '1500785D', 'CHKAQ'}
Removed 0 columns due to too many missing values.
Removed 0 rows due to missing values.

 
 
 Pourquoi rp et ri sont vides ????
0
0


ValueError: Unable to coerce to Series, length must be 1: given 0

xd 
 
 
 xd
